In [1]:
import torch
import torch.nn as nn
from collections import defaultdict
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset
import tqdm
import math
from collections import defaultdict

In [2]:
FILL_IN = "FILL_IN"

In [3]:
from google.colab import drive
drive.mount('/content/drive/')

names = []
names_path = '/content/drive/MyDrive/ADL_HW/names.txt'
with open(names_path, 'r') as f:
    names = f.read().splitlines()

Mounted at /content/drive/


In [4]:
names

['emma',
 'olivia',
 'ava',
 'isabella',
 'sophia',
 'charlotte',
 'mia',
 'amelia',
 'harper',
 'evelyn',
 'abigail',
 'emily',
 'elizabeth',
 'mila',
 'ella',
 'avery',
 'sofia',
 'camila',
 'aria',
 'scarlett',
 'victoria',
 'madison',
 'luna',
 'grace',
 'chloe',
 'penelope',
 'layla',
 'riley',
 'zoey',
 'nora',
 'lily',
 'eleanor',
 'hannah',
 'lillian',
 'addison',
 'aubrey',
 'ellie',
 'stella',
 'natalie',
 'zoe',
 'leah',
 'hazel',
 'violet',
 'aurora',
 'savannah',
 'audrey',
 'brooklyn',
 'bella',
 'claire',
 'skylar',
 'lucy',
 'paisley',
 'everly',
 'anna',
 'caroline',
 'nova',
 'genesis',
 'emilia',
 'kennedy',
 'samantha',
 'maya',
 'willow',
 'kinsley',
 'naomi',
 'aaliyah',
 'elena',
 'sarah',
 'ariana',
 'allison',
 'gabriella',
 'alice',
 'madelyn',
 'cora',
 'ruby',
 'eva',
 'serenity',
 'autumn',
 'adeline',
 'hailey',
 'gianna',
 'valentina',
 'isla',
 'eliana',
 'quinn',
 'nevaeh',
 'ivy',
 'sadie',
 'piper',
 'lydia',
 'alexa',
 'josephine',
 'emery',
 'julia'

In [5]:
# Dictionaries, {idx -> ch} and {ch -> idx}
itos = defaultdict(int)
stoi = defaultdict(int)
# Number of characters used to predict the target character in the MLP Language Model
block_size = 3
# Batch size used in MLP Language Model
batch_size = 32
# Embedding dimension, per character
d_model = 10
# Hidden dimension for RNN and also MLP Language Models
d_h = 200

# START = START token
stoi['.'] = 0
itos[0] = '.'

count = 1
# Loop over all names and create mappings itos and stoi mapping a unique character to a idx
for name in names:
  for ch in name:
    if not ch in stoi:
      stoi[ch] = count
      itos[count] = ch
      count = count + 1

In [6]:
assert len(stoi) == len(itos)
vocab_size = len(stoi)
assert vocab_size == 27

In [7]:
stoi

defaultdict(int,
            {'.': 0,
             'e': 1,
             'm': 2,
             'a': 3,
             'o': 4,
             'l': 5,
             'i': 6,
             'v': 7,
             's': 8,
             'b': 9,
             'p': 10,
             'h': 11,
             'c': 12,
             'r': 13,
             't': 14,
             'y': 15,
             'n': 16,
             'g': 17,
             'z': 18,
             'f': 19,
             'd': 20,
             'u': 21,
             'k': 22,
             'w': 23,
             'q': 24,
             'x': 25,
             'j': 26})

## BiGram Language Model
- Implement the Bigram Language Model
- Get all the relevent counts, then get the train dataset Perplexity
- Use the class notes

In [154]:
# Using the formulas in class, loop over each name and get the parameters
c1 = defaultdict(int)
c2 = defaultdict(int)
c2_sum = defaultdict(int)
for name in open(names_path, 'r'):
    # Lowercase and remove any whitespace at the end
    name = name.lower().strip()
    # Pad with START = '.' and STOP = '.'
    name = '.'+name+'.'
    # Transform to integer
    name = [stoi[ch] for ch in name]
    # Get the counts for Bigrams and Unigrams
    for i in range(len(name)):
      c1[name[i]] += 1

      if i < len(name) - 1:
        bigram = (name[i], name[i + 1])
        c2[bigram] += 1

In [159]:
# Get perplexity

sumneglogp = 0
T = 0
for name in open(names_path, 'r'):
    # Get rid of white space and lowercase
    name = name.lower().strip()
    # Get the length of the word, without padding
    T += len(name)
    # Don't pad the STOP since we are not generating; pad with START
    name = '.' + name
    # Transform to integrs
    name = [stoi[ch] for ch in name]
    # Get the loss -log(p(name)); use that the log of the product is the sum of the logs
    for i in range(len(name) - 1):
      x, y = name[i], name[i + 1]
      p = c2[(x, y)] / c1[x]

      sumneglogp += -torch.log2(torch.tensor(p))
# Print the Perplexity
print('Preplexity: ', torch.pow(2.0, sumneglogp / T).item())

Preplexity:  14.83566951751709


In [10]:
# Generate a random word using this distributon
# When you generate STOP, terminate
name = '.'
while True:
    c = stoi[name[-1]]
    # Make the distribution from c to any other word other than START
    p = []
    for d in range(vocab_size):
        # Use the same indicies as the dictionary we set up
        # Populate p
        p.append(c2[(c,d)]/c1[c])
    #print(p)
    assert len(p) == vocab_size
    # Sample randmly from the probability using torch.Categorical
    c = torch.distributions.Categorical(torch.tensor(p)).sample()
    # Offset by 1 since we want indices [1, 2, ..., vocab_size]
    if c.item() == 0:
        break
    else:
        name += itos[c.item()]
print('Generated name: ' , name[1:])


Generated name:  keltim


## MLP Language Model

- Implement the MLP language model from below
- Look at page 7, Equation (1)
- https://www.jmlr.org/papers/volume3/bengio03a/bengio03a.pdf

In [116]:
# Number of characters used to predict the target character in the MLP Language Model
block_size = 8
# Batch size used in MLP Language Model
batch_size = 64
# Embedding dimension, per character
d_model = 64
# Hidden dimension for RNN and also MLP Language Models
d_h = 512
lr = 0.003

In [117]:
x_data = []
y_data = []
for name in open(names_path, 'r'):
    name = name.lower().strip()
    # Pad with block_size START tokens and 1 STOP token
    name = ''.join(block_size * ['.']) + name + '.'
    # Loop through name and get the (x, y) pairs
    # Add (x, y) to x_data and y_data and make sure you transform to characters
    # Make sure x_data and y_data have integers, use stoi
    for i in range(len(name) - block_size):
      x = name[i: i + block_size]
      y = name[i + block_size]
      x_data.append([stoi[ch] for ch in x])
      y_data.append(stoi[y])

In [118]:
class MLPLanguageModel(nn.Module):
    def __init__(self):
        super().__init__()
        # An embedding for each character; vocab_size of them
        self.e = nn.Embedding(vocab_size, d_model)
        # H; should take in block_size * d_model vector and output d_h
        self.H = nn.Linear(block_size * d_model, d_h)
        # U; should take in d_h vector and output vocab_size
        self.U = nn.Linear(d_h, vocab_size)
        # W; for the skip connection, should take in block_size * d_model and output vocab_size
        self.W = nn.Linear(block_size * d_model, vocab_size, bias=False)

    # x should be (batch_size, block_size)
    def forward(self, x):
        x = self.e(x)
        x = x.view(x.shape[0], -1)
        logits = self.U(torch.tanh(self.H(x))) + self.W(x)
        return logits

In [119]:
x_data[0], y_data[0]

([0, 0, 0, 0, 0, 0, 0, 0], 1)

In [120]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [121]:
# Define a dataloader with x_data and y_data with batch_size
dl = torch.utils.data.DataLoader(
    torch.utils.data.TensorDataset(torch.tensor(x_data), torch.tensor(y_data)),
    batch_size=batch_size,
    shuffle=True
)

In [122]:
for xb, yb in dl:
    assert xb.shape == (batch_size, block_size)
    assert yb.shape == (batch_size,)
    break

In [123]:
from torch import optim
# Define the MLP model and the Adam optimizer learning rate 0.001
model = MLPLanguageModel().to(device)
optimizer = optim.AdamW(model.parameters(), lr = lr)

In [124]:
total_loss = 0
total_ct = 0
total_epochs = 20

loss_fn = torch.nn.CrossEntropyLoss().to(device)

for _ in tqdm.tqdm(range(total_epochs)):
    total_loss = 0
    total_ct = 0
    for xb, yb in dl:
        xb = xb.to(device)
        yb = yb.to(device)
        # Zero the gradients
        optimizer.zero_grad()

        # Get the logits
        logits = model(xb)

        # Compute the loss
        loss = loss_fn(logits, yb)

        # Get the new gradient
        loss.backward()

        # Clip the gradients to max norm 0.1
        # Use clid_grad_norm from torch
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=0.1)

        # Do a gradient update
        optimizer.step()

        # Get the loss for the batch and get the number of batches
        total_loss += loss.item()
        total_ct += 1

        # Print the loss
        if total_ct % 500 == 0:
            print(f"{total_ct} loss: {total_loss / total_ct}")

  0%|          | 0/20 [00:00<?, ?it/s]

500 loss: 2.5611164202690127
1000 loss: 2.489165368437767
1500 loss: 2.4499320187568663
2000 loss: 2.4167115367650984
2500 loss: 2.3940182425022125
3000 loss: 2.375398099541664


  5%|▌         | 1/20 [00:09<02:53,  9.13s/it]

3500 loss: 2.3601437604767934
500 loss: 2.2163252949714662
1000 loss: 2.211215467095375
1500 loss: 2.207372493902842
2000 loss: 2.201061165630817
2500 loss: 2.197271681165695
3000 loss: 2.1934944974184036


 10%|█         | 2/20 [00:18<02:45,  9.17s/it]

...

3500 loss: 1.9770412889208113
500 loss: 1.9449399709701538
1000 loss: 1.9599355936050415
1500 loss: 1.9620868391990662
2000 loss: 1.9657794203758239
2500 loss: 1.9697951674461365
3000 loss: 1.973549701889356


 95%|█████████▌| 19/20 [02:58<00:09,  9.14s/it]

3500 loss: 1.9765898958274297
500 loss: 1.9435881385803222
1000 loss: 1.9522997392416
1500 loss: 1.9539048233826954
2000 loss: 1.9627580479383469
2500 loss: 1.967536694097519
3000 loss: 1.9687123293876647


100%|██████████| 20/20 [03:08<00:00,  9.42s/it]

3500 loss: 1.9720457191467284


In [125]:
with torch.no_grad():
    # Get perplexity
    sumneglogp = 0
    T = 0
    for name in open(names_path, 'r'):
        name = name.lower().strip()
        T += len(name)
        # Pad with block_size START tokens
        name = ''.join(['.'] * block_size) + name
        x_data = []
        y_data = []
        # Gather all the terms over the loss
        # Notice that we compute -log p(...abc)
        # Which is -log p(a | ...) - log p(b | a..) - log p(c | ba.)
        for i in range(len(name) - block_size):
            x = name[i:i+block_size]
            y = name[i+block_size]
            x_data.append([stoi[ch] for ch in x])
            y_data.append(stoi[y])
        # Gather the loss over the name, for each term
        # You need to get the softmax loss for each term
        # Can either use the CrossEntropyLoss or do this manually
        # Compute the loss
        xb = torch.tensor(x_data).to(device)
        yb = torch.tensor(y_data).to(device)
        logits = model(xb)

        # Use reduction "sum" so you don't need to worry about N
        loss = torch.nn.CrossEntropyLoss(reduction="sum")(logits, yb)

        # Change to log base 2
        loss /= torch.log(torch.tensor(2.0))

        sumneglogp += loss

    print('Preplexity: ', torch.pow(2.0, sumneglogp.clone().detach() / T).item())

Preplexity:  8.160454750061035


In [137]:
# Generate a random word using this distributon
# When you generate STOP, terminate
name = ''.join(block_size * ['.'])
while True:
    # Get the idx
    c = torch.tensor([stoi[ch] for ch in name[-block_size:]]).unsqueeze(0).to(device)
    # Make the distribution from c to any other word other than START
    logits = model(c)
    p = torch.softmax(logits, dim=1)
    # Randomly sample from p a new character
    c = torch.distributions.Categorical(p).sample()
    if c.item() == 0:
        break
    else:
        name += itos[c.item()]
print('Generated name: ' , name[block_size:])

Generated name:  harney


## RNN Language Model
- For each name, run an RNN character by character
- Use the recursion x = Tanh()(Wh @ h + Wx @ x + bh + bx) and y = Softmax()(Wy h + by)
- Do not use the RNN Cell from PyTorch, do this manually as hinted below

In [164]:
# Number of characters used to predict the target character in the MLP Language Model
block_size = 8
# Batch size used in MLP Language Model
batch_size = 64
# Embedding dimension, per character
d_model = 64
# Hidden dimension for RNN and also MLP Language Models
d_h = 512
lr = 1e-4

In [165]:
class RNNLanguageModel(nn.Module):
    def __init__(self):
        super().__init__()
        # Each token has an embedding of size vocab_size
        self.e = nn.Embedding(vocab_size, d_model)
        # Wh used to map hidden to hidden
        self.Wh = nn.Linear(d_h, d_h)
        self.Wx = nn.Linear(d_model, d_h)
        self.Wy = nn.Linear(d_h, vocab_size)

    def forward(self, x, h):
        # Run through to get the embedding for the token
        # The embedding per token is the feature vector x  we pass into the
        # Represent x as an embedding
        x = self.e(x)
        # Get the hidden state
        h = torch.tanh(self.Wx(x) + self.Wh(h))
        # Get the logits we use to predict y
        y = self.Wy(h)
        # Return the z predicting y for the timestep we are at and the next hidden state
        return y, h

In [166]:
model = RNNLanguageModel()
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

In [167]:
total_loss = 0
total_ct = 0
total_epochs = 5

loss_fn = nn.CrossEntropyLoss()

for _ in tqdm.tqdm(range(total_epochs)):
    total_loss = 0
    total_ct = 0
    for name in open(names_path, 'r'):
        name = name.lower().strip()
        # Add the start and end padding token
        name = '.' + name + '.'
        # name[:-1]
        x_data = torch.tensor([stoi[c] for c in name[:-1]])
        # name[1:]
        y_data = torch.tensor([stoi[c] for c in name[1:]])
        logits = []
        # Set the hidden state to random
        h = torch.zeros(1, d_h)
        # Zero the grad
        optimizer.zero_grad()

        # Loop through each token and get the new h and then pass it forward
        # Accumulate all the logits
        for x in x_data:
            y, h = model(x.unsqueeze(0), h)

            logits.append(y)

        # Put all the logits into one tensor
        logits = torch.cat(logits, dim=0)

        # Compute the loss
        loss = loss_fn(logits, y_data)

        # Get the new gradient
        loss.backward()

        # Clip the gradients at max norm 0.1
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.1)

        # Do a gradient update
        optimizer.step()

        # Get the loss for the batch and get the number of batches
        total_loss += loss.item()
        total_ct += 1

        if total_ct and total_ct % 100 == 0:
            print(f"{total_ct} loss: {total_loss / total_ct}")


  0%|          | 0/5 [00:00<?, ?it/s]

100 loss: 3.024248788356781
200 loss: 2.7756699472665787
300 loss: 2.6327125302950543
400 loss: 2.5524486884474755
500 loss: 2.495197642326355
600 loss: 2.460022052923838
700 loss: 2.412215451342719
800 loss: 2.376108872294426
900 loss: 2.3537119347519346
1000 loss: 2.3414170775413514
1100 loss: 2.32774516582489
1200 loss: 2.3134298036495844
1300 loss: 2.301156713504058
1400 loss: 2.294263433473451
1500 loss: 2.283769733985265
1600 loss: 2.273359841555357
1700 loss: 2.267136492027956
1800 loss: 2.260103164845043
1900 loss: 2.2556384072805704
2000 loss: 2.247425367832184
2100 loss: 2.245547396909623
2200 loss: 2.241198167042299
2300 loss: 2.2366849151901578
2400 loss: 2.236392797281345
2500 loss: 2.2315450875759124
2600 loss: 2.22719523865443
2700 loss: 2.223278357408665
2800 loss: 2.218814762873309
2900 loss: 2.215516412011508
3000 loss: 2.2101728307008743
3100 loss: 2.2071302610443486
3200 loss: 2.205136521793902
3300 loss: 2.201382686875083
3400 loss: 2.197571717711056
3500 loss: 2.1

 20%|██        | 1/5 [03:49<15:19, 229.75s/it]

32000 loss: 2.1692749907784163
100 loss: 2.316215195655823
200 loss: 2.175766730904579
300 loss: 2.1221226783593496
400 loss: 2.0902100989222525
500 loss: 2.069677738428116
600 loss: 2.0548035273949306
700 loss: 2.03210668189185
800 loss: 2.017814715504646
900 loss: 2.006661128732893
1000 loss: 2.003008593082428
1100 loss: 1.9993785857070576
1200 loss: 1.993178129196167
1300 loss: 1.989428836107254
1400 loss: 1.9914348483937128
1500 loss: 1.986277157306671
1600 loss: 1.98311891913414
1700 loss: 1.9823441082589766
1800 loss: 1.981558616426256
1900 loss: 1.9815525070617073
2000 loss: 1.9785224974155426
2100 loss: 1.9797314051219395
2200 loss: 1.9794742646542463
2300 loss: 1.9791785793719083
2400 loss: 1.982382825712363
2500 loss: 1.9802202798366546
2600 loss: 1.979103046151308
2700 loss: 1.9792557899157206
2800 loss: 1.9779978179080147
2900 loss: 1.9776421807963274
3000 loss: 1.9743256594340006
3100 loss: 1.9748936104774475
3200 loss: 1.9752689960226417
3300 loss: 1.9741230771035858
3400

 40%|████      | 2/5 [07:36<11:24, 228.01s/it]

...

32000 loss: 2.0354336080718785
100 loss: 2.1586719655990603
200 loss: 2.013407570719719
300 loss: 1.9649694085121154
400 loss: 1.9373201274871825
500 loss: 1.922537273168564
600 loss: 1.9104167364041011
700 loss: 1.8938491087300438
800 loss: 1.8843842719495296
900 loss: 1.8744227877590391
1000 loss: 1.8718134320378303
1100 loss: 1.870057117342949
1200 loss: 1.863064178576072
1300 loss: 1.859809053998727
1400 loss: 1.8632038320388113
1500 loss: 1.8599049610296885
1600 loss: 1.8581407311558724
1700 loss: 1.8593858789696414
1800 loss: 1.8601743181877666
1900 loss: 1.8603240004652424
2000 loss: 1.85891953638196
2100 loss: 1.8604802079143978
2200 loss: 1.8609415211731737
2300 loss: 1.8608421130024868
2400 loss: 1.8652355379859606
2500 loss: 1.8629401886463166
2600 loss: 1.8621170325462635
2700 loss: 1.8627938245402442
2800 loss: 1.8625850319862365
2900 loss: 1.8631980790351999
3000 loss: 1.8603467333316803
3100 loss: 1.861727966685449
3200 loss: 1.8631289121881127
3300 loss: 1.8622160802104

 80%|████████  | 4/5 [14:57<03:42, 222.50s/it]

32000 loss: 2.001787384772673
100 loss: 2.096184639930725
200 loss: 1.9577952152490616
300 loss: 1.9146470328172047
400 loss: 1.8900034570693969
500 loss: 1.8762060840129853
600 loss: 1.8647828362385432
700 loss: 1.8507091212272644
800 loss: 1.842885570973158
900 loss: 1.8335737211174434
1000 loss: 1.8313607861995698
1100 loss: 1.8293718166784807
1200 loss: 1.8226569435993831
1300 loss: 1.8197561091643113
1400 loss: 1.8236978632637433
1500 loss: 1.8210105921427409
1600 loss: 1.819961281940341
1700 loss: 1.821612182154375
1800 loss: 1.823024149139722
1900 loss: 1.823404287852739
2000 loss: 1.8225304153561592
2100 loss: 1.824157748733248
2200 loss: 1.8249748174710707
2300 loss: 1.8249196614389833
2400 loss: 1.8294596226265034
2500 loss: 1.827091033244133
2600 loss: 1.8264132437109948
2700 loss: 1.82730337286437
2800 loss: 1.8273962673757758
2900 loss: 1.828188793515337
3000 loss: 1.8255954394141833
3100 loss: 1.8273988133284353
3200 loss: 1.8290126817114651
3300 loss: 1.827938710252444
3

100%|██████████| 5/5 [18:34<00:00, 222.92s/it]

32000 loss: 1.9744793342817575


In [168]:
with torch.no_grad():
    # Get perplexity
    sumneglogp = 0
    T = 0
    for name in open(names_path, 'r'):
        name = name.lower().strip()
        T += len(name)
        name = '.' + name
        # Get the name from index 0 to -1 exclusive end
        x_data = torch.tensor([stoi[c] for c in name[:-1]])
        # Get the y from index 1 to end inclusive end
        y_data = torch.tensor([stoi[c] for c in name[1:]])
        # logits per token prediction
        logits = []
        # Initialize the h vector to random
        h = torch.zeros(1, d_h)
        # Loop over each chracter in the name and pass h and this into the RNN
        # Get the new logit
        for x in x_data:
            # Get the int for x
            x = x.unsqueeze(0)
            # Get z and h
            z, h = model(x, h)
            # Append to logit
            logits.append(z)

        # Get all the logits for each character
        logits = torch.cat(logits, dim=0)

        # Compute the loss across all characters
        loss = torch.nn.CrossEntropyLoss(reduction="sum")(logits, y_data)

        # Change to log base 2
        # log2(x) = ln(x) / ln(2)
        loss /= torch.log(torch.tensor(2))

        sumneglogp += loss.item()

    # sumneglogp is -log(p('.' + name1)) -log(p('.' + name2)) -log(p('.' + name3)) ...
    # Divide by the appropriate term to get the answer we want
    print('Preplexity: ', torch.pow(2, torch.tensor(sumneglogp / T)).item())

Preplexity:  13.036799430847168


In [181]:
# Generate a random word using this distributon
# Intialize the word with
name = '.'
# Initialize h to random
h = torch.zeros(1, d_h)
while True:
    # Make c to an integer
    c = torch.tensor([stoi[name[-1]]])
    # Make the distribution from c to any other word other than START
    logits, h = model(c, h)
    # Get p; use Softmax
    p = torch.softmax(logits, dim=1)
    # Sample from p
    c = torch.multinomial(p, num_samples=1)
    # If we generate '.', stop
    if c.item() == 0:
        break
    else:
        name += itos[c.item()]
print('Generated name: ' , name[1:])

Generated name:  zenyikn
